In [1]:
import os
import json
from collections import Counter

import pandas as pd
import sqlalchemy
import dotenv

dotenv.load_dotenv("../.env")

True

In [2]:
engine = sqlalchemy.create_engine(os.getenv("PGCONN"))

In [11]:
pd.read_sql_query("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'", con=engine)

,table_name
0,dataset
1,example
2,structured_link
3,link
4,structured_input
5,structured_example


In [12]:
dataset = pd.read_sql_table("dataset", con=engine)

# Get the dataset ID for the dataset we want to query
dataset.query("name == 'ume-final'")

,id,name,created,meta,session
9,10,ume-final,1740430466,b'{}',False


In [26]:
examples = pd.read_sql_table("example", con=engine)
content = examples["content"].apply(lambda x: json.loads(x))
df = pd.concat([examples.drop("content", axis=1), pd.json_normalize(content)], axis=1)
df["timestamp"] = pd.to_datetime(df["_timestamp"], unit='s')
df

,id,input_hash,task_hash,idx,question,choice_A,choice_B,choice_C,choice_D,correct_answer,...,_view_id,overall,topic,vocabulary,answer,_timestamp,choices,_annotator_id,_session_id,timestamp
0,1,869596046,-482254096,487,A biologist models the population of bacteria ...,The initial population of bacteria in the petr...,The minimum population the petri dish can sustain,The average population of bacteria over the fi...,The maximum population the petri dish can sustain,D,...,blocks,1,3,0,accept,1740425121,2,ume-catherine,ume-catherine,2025-02-24 19:25:21
1,2,1965004864,-2043517965,591,A savings account starts with \$300 and gains ...,8 weeks,12 weeks,24 weeks,30 weeks,B,...,blocks,2,3,2,accept,1740844196,3,ume-final-catherine,ume-final-catherine,2025-03-01 15:49:56
2,3,-759792225,1689464308,487,A biologist models the population of bacteria ...,The initial population of bacteria in the petr...,The minimum population the petri dish can sustain,The average population of bacteria over the fi...,The maximum population the petri dish can sustain,D,...,blocks,1,2,1,accept,1740847740,3,ume-final-catherine,ume-final-catherine,2025-03-01 16:49:00
3,4,-14046686,128649510,508,"A school club with 80 members, equally divided...",$\frac{12}{80}$,$\frac{12}{35}$,$\frac{12}{23}$,$\frac{12}{20}$,B,...,blocks,2,2,3,accept,1740847927,3,ume-final-catherine,ume-final-catherine,2025-03-01 16:52:07
4,5,70955040,-21803035,560,"Company A offers a starting salary of \$30,000...",Year 2,Year 3,Year 4,Year 5,B,...,blocks,2,3,3,accept,1740849204,3,ume-final-catherine,ume-final-catherine,2025-03-01 17:13:24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,313,-2002344528,1009762778,485,Two groups of runners completed a race. The co...,The mean and standard deviation are both 10 se...,"The mean of Group B is 10 seconds less, and th...","The mean stays the same, and the standard devi...","The mean stays the same, and the standard devi...",B,...,blocks,1,1,3,accept,1741193969,3,ume-final-melissa,ume-final-melissa,2025-03-05 16:59:29
313,314,1468834627,399382778,691,Alex starts with \$20 and saves \$5 each week....,2,1,3,4,A,...,blocks,3,3,3,accept,1741194023,3,ume-final-melissa,ume-final-melissa,2025-03-05 17:00:23
314,315,-835003152,-1574643194,639,A farmer has a rectangular field. When rainfal...,$-2x^2+24x -120$,$-2x^2+24x$,$-2x^2+24x +120$,$-2x^2-96x$,A,...,blocks,1,1,1,accept,1741194196,1,ume-final-melissa,ume-final-melissa,2025-03-05 17:03:16
315,316,-416985603,1622362006,695,Two linear functions $f(x)$ and $h(x)$ are giv...,$-6$,$-2$,$-1$,$2$,B,...,blocks,1,1,1,accept,1741195702,1,ume-final-melissa,ume-final-melissa,2025-03-05 17:28:22


In [18]:
df._annotator_id.value_counts()

_annotator_id
ume-final-catherine    101
ume-final-jennifer      83
ume-final-melissa       80
ume-final-stephanie     52
ume-catherine            1
Name: count, dtype: int64

## Missing Scores?

In [28]:
df[df["topic"].isna()].sample(5)

,id,input_hash,task_hash,idx,question,choice_A,choice_B,choice_C,choice_D,correct_answer,...,_view_id,overall,topic,vocabulary,answer,_timestamp,choices,_annotator_id,_session_id,timestamp
13,14,-645976763,-1898733775,558,A student's average score on history quizzes i...,2:03,15:17,3:02,5:02,C,...,blocks,1,NaN,NaN,accept,1740857787,NaN,ume-final-catherine,ume-final-catherine,2025-03-01 19:36:27
133,131,-693723879,385639400,598,For which value of k does the equation $(k+2)(...,$-9$,$-3$,$3$,$9$,C,...,blocks,1,NaN,NaN,accept,1740955170,NaN,ume-final-catherine,ume-final-catherine,2025-03-02 22:39:30
37,32,2000406845,736212584,593,A researcher wants to study the average time c...,The survey was only conducted on a single soci...,The survey was only accessible to volunteers w...,The survey's sample size was too small to repr...,The survey did not include college students wh...,A,...,blocks,1,NaN,NaN,accept,1740875141,NaN,ume-final-catherine,ume-final-catherine,2025-03-02 00:25:41
125,124,361999317,-274930376,632,A fitness instructor creates personalized work...,$19$,$15$,$20$,$16$,B,...,blocks,1,NaN,NaN,accept,1740953996,NaN,ume-final-catherine,ume-final-catherine,2025-03-02 22:19:56
57,47,1380721861,-1973318730,475,"Given the equation $\frac{1}{t} + kt = m$, whe...",$\frac{m + \sqrt{m^2 - 4k}}{2k}$,$\frac{m \pm \sqrt{m^2 - 4}}{2k}$,$\frac{m \pm \sqrt{m^2 - 4k}}{2k}$,$\frac{m \pm \sqrt{m^2 + 4k}}{2k}$,C,...,blocks,3,NaN,NaN,accept,1740945230,NaN,ume-final-catherine,ume-final-catherine,2025-03-02 19:53:50


# Annotator Overlap

In [32]:
df.groupby("idx").size().value_counts()

2    112
1     78
3      5
Name: count, dtype: int64